# Wavelet

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as scsig

In [ ]:
# Test scipy implementation
M = 100
s = 9.5 # = wavelet scale
w = 5.0 # = number of cycles within the window, NO unit!!
wavelet = scsig.morlet2(M, s, w)
plt.plot(wavelet.real)
plt.plot(wavelet.imag)
plt.grid(True)
plt.show()

In [ ]:
def morlet(t, fc, sigma):
    """
    Parameters
    ----------
        t: array in [s]
        fc: float [Hz]
            center frequency of the modulation [Hz]
        sigma: float [s] 
            standard deviation of Gaussian window, corresponding to the duration 
    """
    print(f'number of cycles in a window w = {2*3* sigma* fc / sigma}')
    # Wavelet
    # (Cf1) Eq. (4.60) to (4.62) in "A wavelet tour of signal processing" by Mallat
    # (Cf2) https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.morlet.html
    # Gaussian window
    g = np.pi**(-0.25)* np.sqrt(1/sigma) * np.exp(-0.5* (t/sigma)**2)
    # Base wavelet: t/sigma ensures that the number of cycles remain the same within the window
    psi = np.exp(1j* 2*np.pi* fc* (t/sigma))* g
    # Zero-mean 
    psi = psi - psi.mean()
    # Normalize, s.t. the energy abs(psi)**2 == 1
    energy = np.sum(np.abs(psi)**2)
    psi = psi / np.sqrt(energy)
    return psi
   

In [ ]:
t, dt = np.linspace(-10.0, 10.0, 1001, retstep=True)
t = t[:-1] # make it the length 100
fs = 1/dt # [Hz]
fc = 1.3 #[Hz]
s = 1.8 # [s], standard deviation of Gaussian, corresponds to the wavelet scale (= width)
fbin = 2*np.pi*fc *fs / (2*s*np.pi) #[Hz], frequency bin of interest ("fundamental frequency")
psi = morlet(t, fc, s)
print(f'fs = {fs}Hz')
print(f'Current bin = {fbin}Hz')

In [ ]:
plt.plot(np.abs(psi))
plt.plot(psi.real)
plt.plot(psi.imag)
plt.show()

In [ ]:
# Scipy
t, dt = np.linspace(0.0, 5.0, 5001, retstep=True)
t = t[:-1]
fs = 1/dt
print(f'fs = {fs}Hz')

w = 7.8
sig = np.cos(2*np.pi*(50 + 10*t)*t) + np.sin(40*np.pi*t)
freq = np.linspace(1, fs/2, 250)
widths = w*fs / (2*freq*np.pi)

In [ ]:
cwtm = scsig.cwt(sig, scsig.morlet2, widths, w=w)

In [ ]:
fig, axs = plt.subplots(1, 3)
axs[0].plot(t[:100], sig[:100])
axs[0].set_title('Time domain')

# Magnitude
# Use pcolormesh to specify the y-axis
axs[1].pcolormesh(t, freq, np.abs(cwtm), cmap='viridis', shading='gouraud')
axs[1].set_xlim(left=0.0)
axs[1].set_ylim(top=200.0)
axs[1].set_title('Magnitude')

# Phase
axs[2].pcolormesh(t, freq, np.angle(cwtm), cmap='viridis', shading='gouraud')
axs[2].set_xlim(left=0.0)
axs[2].set_ylim(top=200.0)
axs[2].set_title('Phase')

plt.show()